In [1]:
# !python

# Import

In [2]:
import mne
from mne.preprocessing import read_ica
import numpy as np
import matplotlib.pyplot as plt

from keras.models import Model
from keras.layers import Dense, Activation, Permute, Dropout, SpatialDropout2D
from keras.layers import Conv2D, MaxPooling2D, AveragePooling2D
from keras.layers import SeparableConv2D, DepthwiseConv2D
from keras.layers import BatchNormalization
from keras.layers import Input, Flatten
from keras.constraints import max_norm
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from keras.utils import np_utils

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from sklearn.metrics import ConfusionMatrixDisplay, f1_score
from imblearn.combine import SMOTEENN
from imblearn.over_sampling import SMOTE

# Def func

In [3]:
data_path = '../input/ern-data/ERN Raw Data and Scripts Only/' #@param {type:"string"}
# data_path = '../input/ern-raw-data-bidscompatible' #@param {type:"string"}

In [4]:
def plot(history):
    # summarize history for accuracy
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'val'], loc='upper left')
    plt.show()
    # summarize history for loss
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'val'], loc='upper left')
    plt.show()

In [5]:
from keras import backend as K
import tensorflow as tf
def focal_loss(gamma=2., alpha=.25):
    def focal_loss_fixed(y_true, y_pred):
        pt_1 = tf.where(tf.equal(y_true, 1), y_pred, tf.ones_like(y_pred))
        pt_0 = tf.where(tf.equal(y_true, 0), y_pred, tf.zeros_like(y_pred))
        return -K.mean(alpha * K.pow(1. - pt_1, gamma) * K.log(pt_1+K.epsilon())) - K.mean((1 - alpha) * K.pow(pt_0, gamma) * K.log(1. - pt_0 + K.epsilon()))
    return focal_loss_fixed

In [6]:
def getData(id, pick_channels=['Fz'], metadata_tmin=-0.2, metadata_tmax=0.6, 
            epochs_tmin=-0.2, epochs_tmax=0.6, l_freq=0.1, h_freq=30,
            baseline=(-0.2, -0.1)):
    
#     id=str(id).zfill(3)

#     raw = mne.io.read_raw(f'{data_path}/{id}/{id}_ERN_shifted_ds_reref_ucbip_hpfilt_ica_weighted.set', preload=True)
#     raw = mne.io.read_raw(f'{data_path}/{id}/{id}_ERN.set', preload=True)

#     raw = mne.io.read_raw(f'{data_path}/sub-{id}/eeg/sub-{id}_task-ERN_eeg.set', preload=True)
    raw = mne.io.read_raw(f'../input/erp-core-ica/{id}_eeg_ica.fif', preload=True)
#     raw = mne.io.read_raw(f'../input/fork-of-erp-core-ica/{id}_eeg_ica.fif', preload=True)



    raw.pick_channels(pick_channels)
    raw.filter(l_freq,h_freq,verbose=0)
    
    raw.resample(128)

    all_events, all_event_id = mne.events_from_annotations(raw)

    c_row_events = list(np.intersect1d([*all_event_id],row_events))

    metadata, events, event_id = mne.epochs.make_metadata(events=all_events, 
                                                          event_id=all_event_id,
                                                          tmin=metadata_tmin, 
                                                          tmax=metadata_tmax, 
                                                          sfreq=raw.info['sfreq'],
                                                          row_events=c_row_events,
                                                          )
    
    metadata['response_correct']=False
    metadata.loc[metadata['event_name'].isin(correct_response), 'response_correct'] = True
    reject = {'eeg': 250e-6}
    epochs = mne.Epochs(raw=raw, tmin=epochs_tmin, tmax=epochs_tmax,
                        baseline=baseline, 
                        events=events, event_id=event_id, metadata=metadata,
                        reject=reject,
                        preload=True,
                        verbose=0)
    
    
    X = epochs.get_data()[2:]
    y = (epochs.metadata.response_correct*1).to_numpy()[2:]

    return X,y

# Parameter

In [7]:
channels1 = ['Fz','Cz']
channels2 = ['FP1', 'F3', 'F7', 'FC3', 'C3', 'C5', 'P3', 'P7', 'P9', 'PO7',
            'PO3', 'O1', 'Oz', 'Pz', 'CPz', 'FP2', 'Fz', 'F4', 'F8', 'FC4',
            'FCz', 'Cz', 'C4', 'C6', 'P4', 'P8', 'P10', 'PO8', 'PO4', 'O2']
channels3 = ['F3', 'F7', 'C3', 'P3', 'P7', 'O1', 'Oz', 'Pz', 'Fz', 'F4', 'F8', 
             'Cz', 'C4', 'P4', 'P8', 'O2']
channels4 = ['F3', 'F7', 'FC3', 'C3', 'C5', 'Fz', 'F4', 'F8', 'FC4',
             'FCz', 'Cz', 'C4', 'C6',]
channels5 = ['F3', 'F7', 'C3', 'O1', 'Oz', 'Pz', 'Fz', 'F4', 'F8', 
             'Cz', 'C4', 'O2']


channels6 = ['Fz','FCz', 'Cz']

channels7= ['Cz', 'Fz', 'F7', 'F3', 'C3', 'C4', 'F4', 'F8']


In [8]:
pick_channel = channels4

In [9]:
""" response - flanker - target
    1: left/compatible
    2: right/incompatible
"""

correct_response = ['111', '121',          # respones: left  - target: left
                    '212', '222',]         # respones: right - target: right

incorrect_response = ['112', '122',         # respones: left  - target: right
                     '211', '221',]         # respones: right - target: left

row_events = correct_response + incorrect_response

# Load data

In [10]:
# len(X)

In [11]:
X=[]
y=[]
id_40 = [1, 3, 4, 6, 8, 10, 12, 13, 15, 16, 18, 21, 22, 27, 28, 29, 30, 32, 35, 37, 38, 39]

for i in range(1,41):
# for i in id_40:
    Xi,yi=getData(id=i, pick_channels=pick_channel, 
#                   l_freq=4, h_freq=8
                 )
    # print(Xi.shape)
    if i==1:
        X=Xi
        y=yi
    else:
        X=np.append(X,Xi,axis=0)
        y=np.append(y,yi,axis=0)


Opening raw data file ../input/erp-core-ica/1_eeg_ica.fif...


/tmp/ipykernel_19/1045735790.py:11: RuntimeWarning: This filename (../input/erp-core-ica/1_eeg_ica.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw = mne.io.read_raw(f'../input/erp-core-ica/{id}_eeg_ica.fif', preload=True)


    Range : 0 ... 935935 =      0.000 ...   913.999 secs
Ready.
Reading 0 ... 935935  =      0.000 ...   913.999 secs...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.1 - 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.10
- Lower transition bandwidth: 0.10 Hz (-6 dB cutoff frequency: 0.05 Hz)
- Upper passband edge: 30.00 Hz
- Upper transition bandwidth: 7.50 Hz (-6 dB cutoff frequency: 33.75 Hz)
- Filter length: 33793 samples (33.001 sec)

Used Annotations descriptions: ['11', '111', '112', '12', '121', '122', '21', '211', '212', '22', '221', '222']
Adding metadata with 14 columns
Replacing existing metadata with 14 columns
402 matching events found
Applying baseline correction (mode: mean)
0 projection items activated
Loading data for 402 events

/tmp/ipykernel_19/1045735790.py:11: RuntimeWarning: This filename (../input/erp-core-ica/2_eeg_ica.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw = mne.io.read_raw(f'../input/erp-core-ica/{id}_eeg_ica.fif', preload=True)


    Range : 0 ... 900095 =      0.000 ...   878.999 secs
Ready.
Reading 0 ... 900095  =      0.000 ...   878.999 secs...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.1 - 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.10
- Lower transition bandwidth: 0.10 Hz (-6 dB cutoff frequency: 0.05 Hz)
- Upper passband edge: 30.00 Hz
- Upper transition bandwidth: 7.50 Hz (-6 dB cutoff frequency: 33.75 Hz)
- Filter length: 33793 samples (33.001 sec)

Used Annotations descriptions: ['11', '111', '112', '12', '121', '122', '21', '212', '22', '221', '222']
Adding metadata with 13 columns
Replacing existing metadata with 13 columns
402 matching events found
Applying baseline correction (mode: mean)
0 projection items activated
Loading data for 402 events and 10

...

    Range : 0 ... 1069055 =      0.000 ...  1043.999 secs
Ready.
Reading 0 ... 1069055  =      0.000 ...  1043.999 secs...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.1 - 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.10
- Lower transition bandwidth: 0.10 Hz (-6 dB cutoff frequency: 0.05 Hz)
- Upper passband edge: 30.00 Hz
- Upper transition bandwidth: 7.50 Hz (-6 dB cutoff frequency: 33.75 Hz)
- Filter length: 33793 samples (33.001 sec)

Used Annotations descriptions: ['11', '111', '112', '12', '121', '122', '21', '211', '212', '22', '221', '222']
Adding metadata with 14 columns
Replacing existing metadata with 14 columns
402 matching events found
Applying baseline correction (mode: mean)
0 projection items activated
Loading data for 402 even

/tmp/ipykernel_19/1045735790.py:11: RuntimeWarning: This filename (../input/erp-core-ica/39_eeg_ica.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw = mne.io.read_raw(f'../input/erp-core-ica/{id}_eeg_ica.fif', preload=True)


    Range : 0 ... 829439 =      0.000 ...   809.999 secs
Ready.
Reading 0 ... 829439  =      0.000 ...   809.999 secs...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.1 - 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.10
- Lower transition bandwidth: 0.10 Hz (-6 dB cutoff frequency: 0.05 Hz)
- Upper passband edge: 30.00 Hz
- Upper transition bandwidth: 7.50 Hz (-6 dB cutoff frequency: 33.75 Hz)
- Filter length: 33793 samples (33.001 sec)

Used Annotations descriptions: ['11', '111', '112', '12', '121', '122', '21', '211', '212', '22', '221', '222']
Adding metadata with 14 columns
Replacing existing metadata with 14 columns
401 matching events found
Applying baseline correction (mode: mean)
0 projection items activated
Loading data for 401 events

/tmp/ipykernel_19/1045735790.py:11: RuntimeWarning: This filename (../input/erp-core-ica/40_eeg_ica.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw = mne.io.read_raw(f'../input/erp-core-ica/{id}_eeg_ica.fif', preload=True)


    Range : 0 ... 899071 =      0.000 ...   877.999 secs
Ready.
Reading 0 ... 899071  =      0.000 ...   877.999 secs...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.1 - 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.10
- Lower transition bandwidth: 0.10 Hz (-6 dB cutoff frequency: 0.05 Hz)
- Upper passband edge: 30.00 Hz
- Upper transition bandwidth: 7.50 Hz (-6 dB cutoff frequency: 33.75 Hz)
- Filter length: 33793 samples (33.001 sec)

Used Annotations descriptions: ['11', '111', '112', '12', '121', '122', '21', '211', '212', '22', '221', '222']
Adding metadata with 14 columns
Replacing existing metadata with 14 columns
398 matching events found
Applying baseline correction (mode: mean)
0 projection items activated
Loading data for 398 events

In [12]:
kernels, chans, samples = 1, X.shape[1], X.shape[-1]

# Pre-Process

# Split train test

In [13]:
from imblearn.under_sampling import RandomUnderSampler, EditedNearestNeighbours

In [14]:
# cc = RandomUnderSampler(
# #     random_state=0
# #                         sampling_strategy=0.5
#                        )
# X_ = X.reshape(len(X),-1)
# X_resampled, y_resampled = cc.fit_resample(X_, y)

In [15]:
# X_resampled.shape

In [16]:
# y_resampled.sum()

In [17]:
def split_data(X_train, X_test, y_train, y_test, type=None):

    if type =='smote':
        smt = SMOTE()
        X_train_ = X_train.reshape(len(X_train),-1,)
        X_train, y_train = smt.fit_resample(X_train_, y_train)
    elif type=='under':
        smt = RandomUnderSampler(
            random_state=123, sampling_strategy=0.5
        )
        X_train_ = X_train.reshape(len(X_train),-1,)
        X_train, y_train = smt.fit_resample(X_train_, y_train)
    
    
    X_train      = X_train.reshape(X_train.shape[0], chans, samples, kernels)
    X_test       = X_test.reshape(X_test.shape[0], chans, samples, kernels)
    
    y_train      = np_utils.to_categorical(y_train)
    y_test       = np_utils.to_categorical(y_test)
    return X_train, X_test, y_train, y_test

# Model

In [18]:
def EEGNet(nb_classes, Chans = 64, Samples = 128, 
             dropoutRate = 0.5, kernLength = 64, F1 = 8, 
             D = 2, F2 = 16, norm_rate = 0.25, dropoutType = Dropout):
    
    input1   = Input(shape = (Chans, Samples, 1))

    ##################################################################
    block1       = Conv2D(F1, (1, kernLength), padding = 'same',
                                   input_shape = (Chans, Samples, 1),
                                   use_bias = False)(input1)
    block1       = BatchNormalization()(block1)
    block1       = DepthwiseConv2D((Chans, 1), use_bias = False, 
                                   depth_multiplier = D,
                                   depthwise_constraint = max_norm(1.))(block1)
    block1       = BatchNormalization()(block1)
    block1       = Activation('elu')(block1)
    block1       = AveragePooling2D((1, 4))(block1)
    block1       = dropoutType(dropoutRate)(block1)
    
    block2       = SeparableConv2D(F2, (1, 16),
                                   use_bias = False, padding = 'same')(block1)
    block2       = BatchNormalization()(block2)
    block2       = Activation('elu')(block2)
    block2       = AveragePooling2D((1, 8))(block2)
    block2       = dropoutType(dropoutRate)(block2)
        
    flatten      = Flatten(name = 'flatten')(block2)
    
    dense        = Dense(nb_classes, name = 'dense', 
                         kernel_constraint = max_norm(norm_rate))(flatten)
    softmax      = Activation('softmax', name = 'softmax')(dense)
    
    return Model(inputs=input1, outputs=softmax)

In [19]:
# def square(x):
#     return K.square(x)

# def log(x):
#     return K.log(K.clip(x, min_value = 1e-7, max_value = 10000))   


# def ShallowConvNet(nb_classes, Chans = 64, Samples = 128, dropoutRate = 0.5)

#     input_main   = Input((Chans, Samples, 1))
#     block1       = Conv2D(40, (1, 13), 
#                                  input_shape=(Chans, Samples, 1),
#                                  kernel_constraint = max_norm(2., axis=(0,1,2)))(input_main)
#     block1       = Conv2D(40, (Chans, 1), use_bias=False, 
#                           kernel_constraint = max_norm(2., axis=(0,1,2)))(block1)
#     block1       = BatchNormalization(epsilon=1e-05, momentum=0.9)(block1)
#     block1       = Activation(square)(block1)
#     block1       = AveragePooling2D(pool_size=(1, 35), strides=(1, 7))(block1)
#     block1       = Activation(log)(block1)
#     block1       = Dropout(dropoutRate)(block1)
#     flatten      = Flatten()(block1)
#     dense        = Dense(nb_classes, kernel_constraint = max_norm(0.5))(flatten)
#     softmax      = Activation('softmax')(dense)
    
#     return Model(inputs=input_main, outputs=softmax)

# Train

In [20]:
import keras.backend as K
def get_f1(y_true, y_pred): #taken from old keras source code
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
    return f1_val

In [21]:
class_weights = {0:1, 1:1}

In [22]:
def train_predict_eegnet(i):
    model = EEGNet(nb_classes=2, Chans = chans, Samples = samples,dropoutRate = 0.5, 
                kernLength = 64, F1 = 8, D = 2, F2 = 16)
    model.compile(
        loss='binary_crossentropy', 
#                   loss=focal_loss(),
                optimizer='adam', 
                metrics = [get_f1]  )

    checkpointer = ModelCheckpoint(filepath=f'./tmp/model{i}.h5', verbose=1,
                                   save_best_only=True)
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5,
                                patience=15, min_lr=0.0001)
    earlyStopping = EarlyStopping(monitor="val_loss", patience=20,
                                restore_best_weights=True,)


    history = model.fit(X_train, y_train, validation_split=0.2,
                        batch_size = 128, 
                        epochs = 200, 
                        verbose = 0,
                        callbacks=[earlyStopping,reduce_lr,checkpointer],
                        class_weight = class_weights)
    return model.predict(X_test)

In [23]:
# def train_predict_shallownet():
#     model = ShallowConvNet(nb_classes=2, Chans = chans, Samples = samples,dropoutRate = 0.5)
    
#     model.compile(loss='binary_crossentropy', 
#                 optimizer='adam', 
#                 metrics = ['accuracy', 'MeanSquaredError',
#         'AUC',])

#     checkpointer = ModelCheckpoint(filepath='./tmp/model.h5', verbose=1,
#                                    save_best_only=True)
#     reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5,
#                                 patience=15, min_lr=0.0001)
#     earlyStopping = EarlyStopping(monitor="val_loss", patience=30,
#                                 restore_best_weights=True,)

#     class_weights = {0:1, 1:1}
#     history = model.fit(X_train, y_train, validation_split=0.2,
#                         batch_size = 256, 
#                         epochs = 300, 
#                         verbose = 0,
#                         callbacks=[earlyStopping,reduce_lr,checkpointer],
#                         class_weight = class_weights)
#     return model.predict(X_test)

In [24]:
def write_report(i):
    with open('readme.txt', 'a') as f:
        
        f.write(f'Lần {i}: ')
        f.write(' '.join(pick_channel))
        f.write('\n EEGNet\n')
        f.write(classification_report(y_true=y_test, y_pred=(pred_eegnet>0.5)))
        
#         f.write('\n ShallowConvNet \n')
#         f.write(classification_report(y_true=y_test, y_pred=(pred_shallow>0.5)))
        
        f.write('\n SVM\n')
        f.write(classification_report(y_pred=pred_svm,y_true=y_test.argmax(axis=1)))

        f.write('\nKNN\n')
        f.write(classification_report(y_pred=pred_knn,y_true=y_test.argmax(axis=1)))
        f.close()

In [25]:
import pandas as pd

In [26]:
d = pd.DataFrame(columns=['EEGNET','SVM', 'KNN','ACC'])

# acc = accuracy_score(y_true=y_test.argmax(axis=1),y_pred=probs.argmax(axis=1))
# f1 = f1_score(y_true=y_test.argmax(axis=1),y_pred=probs.argmax(axis=1),average=None)
# d = d.append({'name':'eegnet', 'f1_0':f1[0], 'f1_1':f1[1], 'acc':acc}, ignore_index=True)
# d

#SVM

In [27]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

In [28]:
from sklearn import decomposition, datasets
from sklearn.pipeline import Pipeline
from sklearn import svm
import joblib


# pca = decomposition.PCA()

# pipe = Pipeline(steps=[('pca', pca), ('svc', svc)])


In [29]:
def train_predict_svm(i):
    svc = svm.SVC(class_weight=class_weights)
    svc.fit(X_train.reshape(len(X_train),-1),y_train.argmax(axis=1))
    y2_pred = svc.predict(X_test.reshape(len(X_test),-1))
#     joblib.dump(svm, f'svm{i}.pkl') 
    # accuracy_score(y_test.argmax(axis=1), y2_pred)
    # ConfusionMatrixDisplay.from_predictions(y1_test,probs,
    #                                     cmap='Blues')
    return y2_pred

#KNN

In [30]:
from sklearn import neighbors

In [31]:
def train_predict_knn(i):
    knn=neighbors.KNeighborsClassifier(n_neighbors=5, 
#                                        weights='distance'
                                      )
    knn.fit(X_train.reshape(len(X_train),-1),y_train.argmax(axis=1))
    y2_pred = knn.predict(X_test.reshape(len(X_test),-1))
#     joblib.dump(knn, f'KNN{i}.pkl') 
    # print(classification_report(y_test.argmax(axis=1), y2_pred))
    # accuracy_score(y_test.argmax(axis=1), y2_pred)
    return y2_pred

In [32]:
from sklearn.model_selection import StratifiedKFold

In [33]:
f1_scores = []
acc = []
rp = []
kfold = StratifiedKFold(n_splits=5, shuffle=False, 
#                         random_state=1234,
                       )
i=1
for train_ids, test_ids in kfold.split(X, y):
    print(i)
    X_train, X_test, y_train, y_test = split_data(X[train_ids], X[test_ids], y[train_ids], y[test_ids], 'under')
    print(len(X_train))
    pred_eegnet = train_predict_eegnet(i)
#     pred_shallow = train_predict_shallownet()

#     f1_scores.append(f1_score(y_test.argmax(axis=1),pred_eegnet.argmax(axis=1), average='macro'))
    pred_svm = train_predict_svm(i)

    pred_knn = train_predict_knn(i)
    d = d.append({
#         'EEGNet':0,
        'EEGNET':f1_score(y_test.argmax(axis=1),pred_eegnet.argmax(axis=1), average='macro'),
#                   'ShallowConvNet':f1_score(y_test.argmax(axis=1),pred_shallow.argmax(axis=1), average='macro'),
                  'SVM':f1_score(y_test.argmax(axis=1),pred_svm, average='macro'), 
                  'KNN':f1_score(y_test.argmax(axis=1),pred_knn, average='macro'),
#                 'SVM':0, 
#                   'KNN':0,
#         'ACC':0,
                 'ACC':accuracy_score(y_true=y_test.argmax(axis=1),y_pred=pred_eegnet.argmax(axis=1))
                 }, ignore_index=True)
    write_report(i)
    i+=1

1
4173


2022-08-01 08:00:49.378530: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.
2022-08-01 08:00:49.634620: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)



Epoch 00001: val_loss improved from inf to 0.67583, saving model to ./tmp/model1.h5

Epoch 00002: val_loss improved from 0.67583 to 0.66212, saving model to ./tmp/model1.h5

Epoch 00003: val_loss improved from 0.66212 to 0.65396, saving model to ./tmp/model1.h5

Epoch 00004: val_loss improved from 0.65396 to 0.64637, saving model to ./tmp/model1.h5

Epoch 00005: val_loss improved from 0.64637 to 0.63896, saving model to ./tmp/model1.h5

Epoch 00006: val_loss improved from 0.63896 to 0.63067, saving model to ./tmp/model1.h5

Epoch 00007: val_loss improved from 0.63067 to 0.61979, saving model to ./tmp/model1.h5

Epoch 00008: val_loss improved from 0.61979 to 0.61322, saving model to ./tmp/model1.h5

Epoch 00009: val_loss improved from 0.61322 to 0.61277, saving model to ./tmp/model1.h5

Epoch 00010: val_loss improved from 0.61277 to 0.60910, saving model to ./tmp/model1.h5

Epoch 00011: val_loss improved from 0.60910 to 0.60353, saving model to ./tmp/model1.h5

Epoch 00012: val_loss di

In [34]:
f = open('readme.txt', 'r')
print(f.read())
f.close()

Lần 1: F3 F7 FC3 C3 C5 Fz F4 F8 FC4 FCz Cz C4 C6
 EEGNet
              precision    recall  f1-score   support

           0       0.57      0.12      0.20       347
           1       0.90      0.99      0.94      2746

   micro avg       0.89      0.89      0.89      3093
   macro avg       0.73      0.56      0.57      3093
weighted avg       0.86      0.89      0.86      3093
 samples avg       0.89      0.89      0.89      3093

 SVM
              precision    recall  f1-score   support

           0       0.57      0.56      0.56       347
           1       0.94      0.95      0.95      2746

    accuracy                           0.90      3093
   macro avg       0.76      0.75      0.75      3093
weighted avg       0.90      0.90      0.90      3093

KNN
              precision    recall  f1-score   support

           0       0.37      0.50      0.43       347
           1       0.93      0.89      0.91      2746

    accuracy                           0.85      3093
   macro

In [35]:
# f = open ("readme.txt", 'w') 
# f.close ()

In [36]:
d

,EEGNET,SVM,KNN,ACC
0,0.572417,0.754811,0.670425,0.891044
1,0.744390,0.741716,0.671480,0.886841
2,0.757568,0.780644,0.649673,0.916586
3,0.729344,0.752234,0.661782,0.896541
4,0.631650,0.767173,0.661418,0.892303
